In [ ]:
import neo
import h5py
import dill
import torch
import pathlib
import numpy as np
import matplotlib.pyplot as plt

import quantities as pq
from config import get_cfg_defaults
from elephant.gpfa import GPFA


# ---- Convert to neo.SpikeTrains ---- #
def array_to_spiketrains(array, bin_size):
    """Convert B x T x N spiking array to list of list of SpikeTrains"""
    stList = []
    for trial in range(len(array)):
        trialList = []
        for channel in range(array.shape[2]):
            times = np.nonzero(array[trial, :, channel])[0]
            counts = array[trial, times, channel].astype(int)
            times = np.repeat(times, counts)
            st = neo.SpikeTrain(times*bin_size*pq.ms, t_stop=array.shape[1]*bin_size*pq.ms)
            trialList.append(st)
        stList.append(trialList)
    return stList

In [1]:
cfg = get_cfg_defaults()
data = h5py.File('data/poisson_obs.h5')

Y = np.array(data['Y'])
X = np.array(data['X'])
C = np.array(data['C'])
b = np.array(data['bias'])

n_trials = 100
bin_size_ms = 5
time_delta = bin_size_ms * 1e-3

n_latents = X.shape[2]
n_neurons = Y.shape[2]
n_time_bins = Y.shape[1]

Y = Y[:n_trials]
X = X[:n_trials]
Y_st_train = array_to_spiketrains(Y, bin_size_ms)


# ---- Run GPFA ---- #
gpfa = GPFA(bin_size=(bin_size_ms * pq.ms), x_dim=n_latents)
gpfa_val_result = gpfa.fit_transform(Y_st_train)

m = gpfa_val_result['latent_variable']
length_scales = gpfa.params_estimated['gamma']

IndentationError: unexpected indent (1430304774.py, line 20)